In [ ]:
import sys
sys.path.append('/root/lib/')
import os
import ROOT as r
import pandas as pd
import json
import shutil

%jsroot on

Welcome to JupyROOT 6.30/02


In [ ]:

shutil.copy('/eos/experiment/milliqan/Configs/mqLumis.json', os.getcwd())
mqLumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')

In [ ]:
def checkBeam(mqLumis, filename):
    run = int(filename.split('Run')[1].split('.')[0])
    file = int(filename.split('.')[1].split('_')[0])
    return mqLumis['beam'].loc[(mqLumis['run'] == run) & (mqLumis['file'] == file)].values[0]

In [ ]:
#create root TChain of files for a given run

run = 1580
dataDir = '/store/user/milliqan/trees/v34/1500/'

#create tchain that will read in tree with name t
mychain = r.TChain('t') 

#add files to mychain
for filename in os.listdir(dataDir):
    if not filename.endswith('.root'): continue
    if 'Run{}'.format(run) in filename:
        print("Adding file {} to the chain".format(filename))
        if checkBeam(mqLumis, filename):
            mychain.Add(dataDir+filename)

#print the number of entries in my tchain
entries = mychain.GetEntries()
print("There are {} total entries in the chain".format(entries))


Adding file MilliQan_Run1580.100_v34.root to the chain
Adding file MilliQan_Run1580.101_v34.root to the chain
Adding file MilliQan_Run1580.102_v34.root to the chain
Adding file MilliQan_Run1580.103_v34.root to the chain
Adding file MilliQan_Run1580.104_v34.root to the chain
Adding file MilliQan_Run1580.105_v34.root to the chain
Adding file MilliQan_Run1580.106_v34.root to the chain
Adding file MilliQan_Run1580.107_v34.root to the chain
Adding file MilliQan_Run1580.108_v34.root to the chain
Adding file MilliQan_Run1580.109_v34.root to the chain
Adding file MilliQan_Run1580.10_v34.root to the chain
Adding file MilliQan_Run1580.110_v34.root to the chain
Adding file MilliQan_Run1580.111_v34.root to the chain
Adding file MilliQan_Run1580.112_v34.root to the chain
Adding file MilliQan_Run1580.113_v34.root to the chain
Adding file MilliQan_Run1580.114_v34.root to the chain
Adding file MilliQan_Run1580.115_v34.root to the chain
Adding file MilliQan_Run1580.116_v34.root to the chain
Adding file

In [ ]:
#select only the events with hits in front and back slab with area > 100k


mychain.Draw(">>eList", "MaxIf$(area, chan==74) > 100000 && MaxIf$(area, chan==75) > 100000", "entryList")
eList = r.gDirectory.Get("eList")
mychain.SetEntryList(eList)

In [ ]:
#create a canvas and draw the time difference 
c1 = r.TCanvas("c1", "c1", 600, 600)
mychain.Draw("MaxIf$(timeFit_module_calibrated, chan==75&&area==MaxIf$(area,chan==75)) - MaxIf$(timeFit_module_calibrated, chan==74&&area==MaxIf$(area,chan==74)) >> h_timeDiff")
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [15]:
#save the output in a root file

#get all files from the gDirectory
h_timeDiff = r.gDirectory.Get('h_timeDiff')

#create the output root file
fout = r.TFile("timingCorrectionsOutput.root", 'RECREATE')

#switch to our root file for writing
fout.cd()

#write the plot to the file
h_timeDiff.Write()

#close the file
fout.Close()

